In [1]:
import os
import random
import time

import imageio.v2 as imageio
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torchvision.utils import make_grid, save_image

# Import checkpoint utilities
from Model import save_checkpoint_generic, load_checkpoint_generic

In [2]:
# # DCGAN Training Setup
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#
# batch_size = 512
# image_size = 64
# nc = 3
# noise_size = 100
# num_epochs = 500
# lr = 0.0002
# beta1 = 0.5
#
# # Transforms
# transform = transforms.Compose([
#     transforms.Resize(image_size),
#     transforms.CenterCrop(image_size),
#     transforms.ToTensor(),
#     transforms.Normalize([0.5] * nc, [0.5] * nc),
# ])
#
# from Model import DCGAN_Dataset
# data_root = "./Dataset/tpdne_faces"
# out_dir = "./output_dcgan"
# if not os.path.exists(data_root):
#     raise FileNotFoundError("Dataset not found")
# os.makedirs(out_dir, exist_ok=True)
#
# dataset = DCGAN_Dataset(data_root, transform=transform)
# dataloader = torch.utils.data.DataLoader(
#     dataset,
#     # batch_size=batch_size,
#     batch_size=len(dataset),
#     shuffle=True,
#     num_workers=0,
#     drop_last=False,
#     pin_memory=True
# )
#
# # seed
# manualSeed = 999
# random.seed(manualSeed)
# torch.manual_seed(manualSeed)
#
# # Initialize models
# from Model import DCGAN_Generator, DCGAN_Discriminator
# generator = DCGAN_Generator(z_dim=noise_size, img_channels=nc).to(device)
# discriminator = DCGAN_Discriminator(img_channels=nc).to(device)
#
# criterionG = nn.BCELoss()
# criterionD = nn.BCELoss()
#
# optimizerG = optim.Adam(generator.parameters(), lr=lr, betas=(beta1, 0.999), foreach=False)
# optimizerD = optim.Adam(discriminator.parameters(), lr=lr, betas=(beta1, 0.999), foreach=False)
#
# # Training config to save with checkpoints
# training_config = {
#     'batch_size': batch_size,
#     'image_size': image_size,
#     'nc': nc,
#     'noise_size': noise_size,
#     'num_epochs': num_epochs,
#     'lr': lr,
#     'beta1': beta1,
#     'manualSeed': manualSeed,
# }


In [3]:
# # TRAIN LOOP DCGAN
# torch.cuda.empty_cache()
#
# # Initialize variables first
# start_epoch = 0
# G_losses = []
# D_losses = []
# loaded_noise = None
#
# real_label = 1.
# fake_label = 0.
# img_list = []
#
# # Load checkpoint (if exists)
# checkpoint = load_checkpoint_generic(out_dir, device)
# # checkpoint = load_checkpoint_generic_compat(out_dir, device)
# if checkpoint:
#     generator.load_state_dict(checkpoint['generator'])
#     discriminator.load_state_dict(checkpoint['discriminator'])
#     optimizerG.load_state_dict(checkpoint['optimizer_G'])
#     optimizerD.load_state_dict(checkpoint['optimizer_D'])
#     start_epoch = checkpoint['epoch']
#     G_losses = checkpoint.get('G_losses', [])
#     D_losses = checkpoint.get('D_losses', [])
#     loaded_noise = checkpoint.get('fixed_noise')
#     config = checkpoint.get('config', {})
#
# # Initialize or use loaded fixed noise
# if loaded_noise is not None:
#     fixed_noise = loaded_noise.to(device)
#     print("Using loaded fixed_noise from checkpoint")
# else:
#     fixed_noise = torch.randn(64, noise_size, 1, 1, device=device)
#     print("Created new fixed_noise for visualization")
#
# print("Starting Training Loop on device:", device)
# print(f"Training from epoch {start_epoch + 1} to {num_epochs}")
# iters = len(G_losses)
# start_time = time.time()
#
# for epoch in range(start_epoch + 1, num_epochs + 1):
#     epoch_start = time.time()
#
#     # ✅ Set models to training mode
#     generator.train()
#     discriminator.train()
#
#     for i, data in enumerate(dataloader, 0):
#
#         ############################
#         # (1) Update Discriminator
#         ############################
#         discriminator.zero_grad()
#
#         real_images = data.to(device)
#         b_size = real_images.size(0)
#         label = torch.full((b_size,), real_label, dtype=torch.float, device=device)
#
#         output = discriminator(real_images).view(-1)
#         errD_real = criterionD(output, label)
#         errD_real.backward()
#         D_x = output.mean().item()
#
#         noise = torch.randn(b_size, noise_size, 1, 1, device=device)
#         fake = generator(noise)
#         label.fill_(fake_label)
#
#         output = discriminator(fake.detach()).view(-1)
#         errD_fake = criterionD(output, label)
#         errD_fake.backward()
#         D_G_z1 = output.mean().item()
#
#         errD = errD_real + errD_fake
#         optimizerD.step()
#
#         ############################
#         # (2) Update Generator
#         ############################
#         generator.zero_grad()
#         label.fill_(real_label)
#
#         output = discriminator(fake).view(-1)
#         errG = criterionG(output, label)
#         errG.backward()
#         D_G_z2 = output.mean().item()
#
#         optimizerG.step()
#
#         G_losses.append(errG.item())
#         D_losses.append(errD.item())
#
#         if i % 50 == 0:
#             print(f"Epoch [{epoch}/{num_epochs}] Batch [{i}/{len(dataloader)}] "
#                   f"Loss_D: {errD.item():.4f} Loss_G: {errG.item():.4f} "
#                   f"D(x): {D_x:.4f} D(G(z)): {D_G_z1:.4f}/{D_G_z2:.4f}")
#
#         iters += 1
#
#     generator.eval()
#
#     # End of epoch: save sample grid
#     with torch.no_grad():
#         fake_fixed = generator(fixed_noise).detach().cpu()
#
#     grid = make_grid(fake_fixed, nrow=8, normalize=True, scale_each=True)
#     epoch_img_path = os.path.join(out_dir, f"epoch_{epoch:04d}.png")
#     save_image(grid, epoch_img_path)
#     img_list.append(epoch_img_path)
#
#     # ✅ Clear memory only after epoch (not every batch)
#     del fake_fixed, grid
#     torch.cuda.empty_cache()
#
#     epoch_time = time.time() - epoch_start
#     print(f"End epoch {epoch}/{num_epochs}  time: {epoch_time:.1f}s  saved: {epoch_img_path}")
#
#     if epoch % 10 == 0 or epoch == num_epochs:
#         # Save
#         save_checkpoint_generic(out_dir, epoch, {
#             'generator': generator.state_dict(),
#             'discriminator': discriminator.state_dict(),
#             'optimizer_G': optimizerG.state_dict(),
#             'optimizer_D': optimizerD.state_dict(),
#             'G_losses': G_losses,
#             'D_losses': D_losses,
#             'fixed_noise': fixed_noise,
#             'config': training_config,
#         })
#
#
# total_time = time.time() - start_time
# print(f"Training finished in {total_time / 60:.2f} minutes.")

In [4]:
# # CycleGAN Training Setup
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#
# # dataset_root = "./Dataset/apple2orange"
# dataset_root = "./Dataset/Summer2Winter_Yosemite"
# data_root_A = os.path.join(dataset_root, "trainA")
# data_root_B = os.path.join(dataset_root, "trainB")
# test_root_A = os.path.join(dataset_root, "testA")
# test_root_B = os.path.join(dataset_root, "testB")
# # data_root_A = "./Dataset/apple2orange/trainA"
# # data_root_B = "./Dataset/apple2orange/trainB"
# # test_root_A = "./Dataset/apple2orange/testA"
# # test_root_B = "./Dataset/apple2orange/testB"
#
# # out_dir = "output_cyclegan_a2o"
# out_dir = "output_cyclegan_s2w"
# image_size = 256
# batch_size = 4
# num_epochs = 50
# lr = 0.0002
# beta1 = 0.5
# lambda_cycle = 10.0
# lambda_identity = 5.0
#
# # Transforms
# transform = transforms.Compose([
#     transforms.Resize((image_size, image_size)),
#     transforms.ToTensor(),
#     transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]),
# ])
#
# from Model import CycleGAN_Dataset
#
# os.makedirs(out_dir, exist_ok=True)
#
# # Training dataset
# dataset = CycleGAN_Dataset(data_root_A, data_root_B, transform=transform)
# dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=2)
#
# # Test dataset (fixed samples)
# test_dataset = CycleGAN_Dataset(test_root_A, test_root_B, transform=transform)
# test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=8, shuffle=False)
#
# # Seed
# manualSeed = 999
# random.seed(manualSeed)
# torch.manual_seed(manualSeed)
#
# # Initialize models
# from Model import CycleGAN_Generator, CycleGAN_Discriminator
#
# G_A2B = CycleGAN_Generator().to(device)
# G_B2A = CycleGAN_Generator().to(device)
# D_A = CycleGAN_Discriminator().to(device)
# D_B = CycleGAN_Discriminator().to(device)
#
# # Loss functions
# criterion_GAN = nn.MSELoss()
# criterion_cycle = nn.L1Loss()
# criterion_identity = nn.L1Loss()
#
# # Optimizers
# optimizer_G = optim.Adam(
#     list(G_A2B.parameters()) + list(G_B2A.parameters()),
#     lr=lr, betas=(beta1, 0.999)
# )
# optimizer_D_A = optim.Adam(D_A.parameters(), lr=lr, betas=(beta1, 0.999))
# optimizer_D_B = optim.Adam(D_B.parameters(), lr=lr, betas=(beta1, 0.999))
#
# # Training config
# training_config = {
#     'batch_size': batch_size,
#     'image_size': image_size,
#     'num_epochs': num_epochs,
#     'lr': lr,
#     'beta1': beta1,
#     'lambda_cycle': lambda_cycle,
#     'lambda_identity': lambda_identity,
#     'manualSeed': manualSeed,
# }

In [5]:
# # Training loop CycleGAN
# torch.cuda.empty_cache()
#
# start_epoch = 0
# G_losses = []
# D_A_losses = []
# D_B_losses = []
# test_A = None
# test_B = None
#
# # Load checkpoint
# checkpoint = load_checkpoint_generic(out_dir, device)
# if checkpoint:
#     G_A2B.load_state_dict(checkpoint['G_A2B'])
#     G_B2A.load_state_dict(checkpoint['G_B2A'])
#     D_A.load_state_dict(checkpoint['D_A'])
#     D_B.load_state_dict(checkpoint['D_B'])
#     optimizer_G.load_state_dict(checkpoint['optimizer_G'])
#     optimizer_D_A.load_state_dict(checkpoint['optimizer_D_A'])
#     optimizer_D_B.load_state_dict(checkpoint['optimizer_D_B'])
#     start_epoch = checkpoint['epoch']
#     G_losses = checkpoint.get('G_losses', [])
#     D_A_losses = checkpoint.get('D_A_losses', [])
#     D_B_losses = checkpoint.get('D_B_losses', [])
#     test_A = checkpoint.get('test_A')
#     test_B = checkpoint.get('test_B')
#     config = checkpoint.get('config', {})
#
# # Get fixed test samples (or use loaded ones)
# if test_A is None or test_B is None:
#     test_A, test_B = next(iter(test_loader))
#     test_A = test_A.to(device)
#     test_B = test_B.to(device)
#     print("Created new fixed test samples")
# else:
#     test_A = test_A.to(device)
#     test_B = test_B.to(device)
#     print("Using loaded fixed test samples from checkpoint")
#
# print(f"Starting CycleGAN training on {device}")
# print(f"Training from epoch {start_epoch + 1} to {num_epochs}")
# start_time = time.time()
#
# for epoch in range(start_epoch + 1, num_epochs + 1):
#     epoch_start = time.time()
#
#     G_A2B.train()
#     G_B2A.train()
#     D_A.train()
#     D_B.train()
#
#     for i, (real_A, real_B) in enumerate(dataloader):
#         batch_start = time.time()
#         real_A = real_A.to(device)
#         real_B = real_B.to(device)
#
#         batch_size_curr = real_A.size(0)
#         real_label = torch.ones(batch_size_curr, 1, 30, 30, device=device)
#         fake_label = torch.zeros(batch_size_curr, 1, 30, 30, device=device)
#
#         # =====================================
#         # Train Generators
#         # =====================================
#         optimizer_G.zero_grad()
#
#         # Identity loss
#         identity_A = G_B2A(real_A)
#         loss_identity_A = criterion_identity(identity_A, real_A)
#
#         identity_B = G_A2B(real_B)
#         loss_identity_B = criterion_identity(identity_B, real_B)
#
#         # GAN loss
#         fake_B = G_A2B(real_A)
#         pred_fake_B = D_B(fake_B)
#         loss_GAN_A2B = criterion_GAN(pred_fake_B, real_label)
#
#         fake_A = G_B2A(real_B)
#         pred_fake_A = D_A(fake_A)
#         loss_GAN_B2A = criterion_GAN(pred_fake_A, real_label)
#
#         # Cycle consistency loss
#         recovered_A = G_B2A(fake_B)
#         loss_cycle_A = criterion_cycle(recovered_A, real_A)
#
#         recovered_B = G_A2B(fake_A)
#         loss_cycle_B = criterion_cycle(recovered_B, real_B)
#
#         # Total generator loss
#         loss_G = (
#                 loss_GAN_A2B + loss_GAN_B2A +
#                 lambda_cycle * (loss_cycle_A + loss_cycle_B) +
#                 lambda_identity * (loss_identity_A + loss_identity_B)
#         )
#
#         loss_G.backward()
#         optimizer_G.step()
#
#         # =====================================
#         # Train Discriminator A
#         # =====================================
#         optimizer_D_A.zero_grad()
#
#         pred_real_A = D_A(real_A)
#         loss_D_real_A = criterion_GAN(pred_real_A, real_label)
#
#         pred_fake_A = D_A(fake_A.detach())
#         loss_D_fake_A = criterion_GAN(pred_fake_A, fake_label)
#
#         loss_D_A = loss_D_real_A + loss_D_fake_A
#         loss_D_A.backward()
#         optimizer_D_A.step()
#
#         # =====================================
#         # Train Discriminator B
#         # =====================================
#         optimizer_D_B.zero_grad()
#
#         pred_real_B = D_B(real_B)
#         loss_D_real_B = criterion_GAN(pred_real_B, real_label)
#
#         pred_fake_B = D_B(fake_B.detach())
#         loss_D_fake_B = criterion_GAN(pred_fake_B, fake_label)
#
#         loss_D_B = loss_D_real_B + loss_D_fake_B
#         loss_D_B.backward()
#         optimizer_D_B.step()
#
#         # Track losses
#         G_losses.append(loss_G.item())
#         D_A_losses.append(loss_D_A.item())
#         D_B_losses.append(loss_D_B.item())
#
#         batch_time = time.time() - batch_start
#         print(f"Epoch [{epoch}/{num_epochs}] Batch [{i}/{len(dataloader)}] Time: {batch_time:.2f}s")
#
#         if i % 1 == 0:
#             print(f"Loss_G: {loss_G.item():.4f} "
#                   f"Loss_D_A: {loss_D_A.item():.4f} "
#                   f"Loss_D_B: {loss_D_B.item():.4f}")
#
#     # Save sample images at end of epoch
#     G_A2B.eval()
#     G_B2A.eval()
#
#     with torch.no_grad():
#         fake_B_sample = G_A2B(test_A)
#         fake_A_sample = G_B2A(test_B)
#
#         comparison = torch.cat([test_A, fake_B_sample, test_B, fake_A_sample])
#         grid = make_grid(comparison, nrow=8, normalize=True, scale_each=True)
#         save_image(grid, os.path.join(out_dir, f"epoch_{epoch:04d}.png"))
#
#     epoch_time = time.time() - epoch_start
#     print(f"Epoch {epoch}/{num_epochs} completed in {epoch_time:.1f}s")
#
#     # Save checkpoint
#     if epoch % 1 == 0 or epoch == num_epochs:
#         save_checkpoint_generic(out_dir, epoch, {
#             'G_A2B': G_A2B.state_dict(),
#             'G_B2A': G_B2A.state_dict(),
#             'D_A': D_A.state_dict(),
#             'D_B': D_B.state_dict(),
#             'optimizer_G': optimizer_G.state_dict(),
#             'optimizer_D_A': optimizer_D_A.state_dict(),
#             'optimizer_D_B': optimizer_D_B.state_dict(),
#             'G_losses': G_losses,
#             'D_A_losses': D_A_losses,
#             'D_B_losses': D_B_losses,
#             'test_A': test_A.cpu(),
#             'test_B': test_B.cpu(),
#             'config': training_config,
#         })
#     torch.cuda.empty_cache()
#
# total_time = time.time() - start_time
# print(f"Training finished in {total_time / 60:.2f} minutes")

In [10]:
# NST

import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torchvision.utils import save_image
from PIL import Image

# ---------------- CONFIG ----------------
output_dir = "output_nst"
input_dir  = "Dataset/NST"
# style_image_path   = os.path.join(input_dir, "nst-picasso.jpg")
style_image_path   = os.path.join(input_dir, "nst-kadinsky.jpg")
content_image_path = os.path.join(input_dir, "nst-dancing.jpg")
# content_image_path = os.path.join(input_dir, "nst-labrador.jpg")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
os.makedirs(output_dir, exist_ok=True)

# ---------------- PREPROCESS / POSTPROCESS ----------------
imagenet_mean = [0.485, 0.456, 0.406]
imagenet_std  = [0.229, 0.224, 0.225]

preprocess = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.ToTensor(),
    transforms.Normalize(mean=imagenet_mean, std=imagenet_std)
])

# When saving, undo normalization and clamp to [0,1]
def denormalize(tensor):
    # tensor shape: (1,3,H,W)
    t = tensor.clone().detach().cpu()
    for c, (m, s) in enumerate(zip(imagenet_mean, imagenet_std)):
        t[:, c, :, :] = t[:, c, :, :] * s + m
    t = torch.clamp(t, 0.0, 1.0)
    return t

def load_img(path, size=512):
    img = Image.open(path).convert("RGB")
    return preprocess(img).unsqueeze(0).to(device)

# ---------------- LOAD IMAGES ----------------
content_img = load_img(content_image_path)
style_img   = load_img(style_image_path)

# ---------------- VGG19 SETUP ----------------
vgg = models.vgg19(weights=models.VGG19_Weights.IMAGENET1K_V1).features
vgg = vgg.to(device).eval()

# FIX: Important for LBFGS — disable inplace ReLU!
for i, layer in enumerate(vgg):
    if isinstance(layer, nn.ReLU):
        vgg[i] = nn.ReLU(inplace=False)

for p in vgg.parameters():
    p.requires_grad_(False)

# mapping: actual layer indices in torchvision VGG19.features
layer_mapping = {
    0:  "conv1_1",
    5:  "conv2_1",
    10: "conv3_1",
    19: "conv4_1",
    21: "conv4_2",   # content layer
    28: "conv5_1"
}

style_layers   = ["conv1_1", "conv2_1", "conv3_1", "conv4_1", "conv5_1"]
content_layers = ["conv4_2"]

# Wrap vgg into sequential 'model' (same as vgg but easier to enumerate)
model = nn.Sequential(*list(vgg))

# ----------------- GRAM -----------------
def gram_matrix(t):
    b, c, h, w = t.size()
    f = t.view(b, c, h * w)
    g = torch.bmm(f, f.transpose(1, 2))     # (b, c, c)
    return g / (c * h * w)

# ---------------- EXTRACT TARGETS ----------------
style_targets = {}
content_targets = {}

# Forward pass through all layers and capture at indices
x = style_img.clone()
y = content_img.clone()

for idx, layer in enumerate(model):
    x = layer(x)
    y = layer(y)
    if idx in layer_mapping:
        name = layer_mapping[idx]
        if name in style_layers:
            style_targets[name] = gram_matrix(x).detach()
        if name in content_layers:
            content_targets[name] = y.detach()

print("Captured style target layers:", list(style_targets.keys()))
print("Captured content target layers:", list(content_targets.keys()))

# ---------------- TRAINABLE IMAGE ----------------
input_img = content_img.clone().requires_grad_(True)

# ---------------- CHECKPOINT DIR ----------------
content_name = os.path.splitext(os.path.basename(content_image_path))[0]
style_name   = os.path.splitext(os.path.basename(style_image_path))[0]
checkpoint_dir = os.path.join(output_dir, f"{content_name}__{style_name}")
os.makedirs(checkpoint_dir, exist_ok=True)
print("Checkpoints saved to:", checkpoint_dir)

# ---------------- OPTIMIZER / HYPERPARAMS ----------------
style_weight = 1e4
content_weight = 1.0
max_steps = 1000

# LBFGS (set max_iter to control upper bound)
optimizer = optim.LBFGS([input_img], lr=1.0, max_iter=max_steps)

save_steps = {50, 100, 150, 200, 250, 300}
step_counter = [0]

# ---------------- TRAINING LOOP ----------------
print("Starting style transfer...")

def closure():
    with torch.enable_grad():
        optimizer.zero_grad()
        x = input_img
        style_loss = 0.0
        content_loss = 0.0

        # forward through full model, check mapping at indices
        for idx, layer in enumerate(model):
            x = layer(x)
            if idx in layer_mapping:
                name = layer_mapping[idx]
                if name in content_layers:
                    content_loss = content_loss + nn.functional.mse_loss(x, content_targets[name])
                if name in style_layers:
                    g = gram_matrix(x)
                    style_loss = style_loss + nn.functional.mse_loss(g, style_targets[name])

        loss = content_weight * content_loss + style_weight * style_loss
        loss.backward()

        step_counter[0] += 1
        s = step_counter[0]

        if s % 50 == 0:
            print(f"[{s:3d}] Content={content_loss.item():.6f}, Style={style_loss.item():.6f}")
            out = denormalize(input_img.detach())
            save_image(out, os.path.join(checkpoint_dir, f"step_{s}.png"))
            print("Saved:", os.path.join(checkpoint_dir, f"step_{s}.png"))

        return loss

# optimizer.step(closure)
#
# # ---------------- FINAL ----------------
# final_out = denormalize(input_img.detach())
# save_image(final_out, os.path.join(checkpoint_dir, "final.png"))
# print("Finished. Final image:", os.path.join(checkpoint_dir, "final.png"))


Captured style target layers: ['conv1_1', 'conv2_1', 'conv3_1', 'conv4_1', 'conv5_1']
Captured content target layers: ['conv4_2']
Checkpoints saved to: output_nst\nst-dancing__nst-kadinsky
Starting style transfer...
[ 50] Content=20.755108, Style=0.001997
Saved: output_nst\nst-dancing__nst-kadinsky\step_50.png
[100] Content=15.333860, Style=0.001262
Saved: output_nst\nst-dancing__nst-kadinsky\step_100.png
[150] Content=13.460974, Style=0.001040
Saved: output_nst\nst-dancing__nst-kadinsky\step_150.png
[200] Content=12.443685, Style=0.000938
Saved: output_nst\nst-dancing__nst-kadinsky\step_200.png
[250] Content=11.852937, Style=0.000882
Saved: output_nst\nst-dancing__nst-kadinsky\step_250.png
[300] Content=11.456408, Style=0.000846
Saved: output_nst\nst-dancing__nst-kadinsky\step_300.png
[350] Content=11.180655, Style=0.000821
Saved: output_nst\nst-dancing__nst-kadinsky\step_350.png
[400] Content=10.974371, Style=0.000802
Saved: output_nst\nst-dancing__nst-kadinsky\step_400.png
[450] Con

In [12]:
# ---------------- BATCH PROCESSING ----------------
import random

def run_style_transfer(content_path, style_path, output_base_dir="output_nst",
                       batch_name="batch", steps=1000, checkpoint_interval=50,
                       style_weight_val=1e4, content_weight_val=1.0):
    """
    Run style transfer on a single content/style pair with checkpoint saving.

    Args:
        content_path: Path to content image
        style_path: Path to style image
        output_base_dir: Base output directory (e.g., "output_nst")
        batch_name: Batch subfolder name (e.g., "batch")
        steps: Maximum optimization steps
        checkpoint_interval: Save checkpoint every N steps (default: 50)
        style_weight_val: Weight for style loss
        content_weight_val: Weight for content loss
    """

    # Load images
    content = load_img(content_path)
    style = load_img(style_path)

    # Extract style targets
    style_targets_new = {}
    x = style.clone()
    for idx, layer in enumerate(model):
        x = layer(x)
        if idx in layer_mapping:
            name = layer_mapping[idx]
            if name in style_layers:
                style_targets_new[name] = gram_matrix(x).detach()

    # Extract content targets
    content_targets_new = {}
    y = content.clone()
    for idx, layer in enumerate(model):
        y = layer(y)
        if idx in layer_mapping:
            name = layer_mapping[idx]
            if name in content_layers:
                content_targets_new[name] = y.detach()

    # Setup checkpoint directory: output_nst/batch/content__style/
    c_name = os.path.splitext(os.path.basename(content_path))[0]
    s_name = os.path.splitext(os.path.basename(style_path))[0]
    ckpt_dir = os.path.join(output_base_dir, batch_name, f"{c_name}__{s_name}")
    os.makedirs(ckpt_dir, exist_ok=True)

    # Initialize from content image
    input_img = content.clone().requires_grad_(True)
    optimizer = optim.LBFGS([input_img], lr=1.0, max_iter=steps)
    step_counter = [0]

    print(f"\nProcessing: {c_name} × {s_name}")
    print(f"Output directory: {ckpt_dir}")

    # Closure function with checkpoint saving
    def closure():
        with torch.enable_grad():
            optimizer.zero_grad()
            x = input_img
            style_loss = 0.0
            content_loss = 0.0

            for idx, layer in enumerate(model):
                x = layer(x)
                if idx in layer_mapping:
                    name = layer_mapping[idx]
                    if name in content_layers:
                        content_loss = content_loss + nn.functional.mse_loss(x, content_targets_new[name])
                    if name in style_layers:
                        g = gram_matrix(x)
                        style_loss = style_loss + nn.functional.mse_loss(g, style_targets_new[name])

            loss = content_weight_val * content_loss + style_weight_val * style_loss
            loss.backward()

            step_counter[0] += 1
            s = step_counter[0]

            # Save checkpoints at specified intervals
            if s % checkpoint_interval == 0:
                print(f"[{s:4d}] Content={content_loss.item():.6f}, Style={style_loss.item():.6f}")
                out = denormalize(input_img.detach())
                save_image(out, os.path.join(ckpt_dir, f"step_{s:04d}.png"))

            return loss

    optimizer.step(closure)

    # Save final result
    final = denormalize(input_img.detach())
    save_image(final, os.path.join(ckpt_dir, "final.png"))
    print(f"✓ Saved final image: {ckpt_dir}/final.png")

    return final


# ---------------- BATCH RUNNER WITH RANDOM STYLE SELECTION ----------------
def run_batch_style_transfer(content_images, style_images, output_base_dir="output_nst",
                            batch_name="batch", steps=500, checkpoint_interval=50):
    """
    Run style transfer for multiple content images with random style selection.

    Args:
        content_images: List of content image paths
        style_images: List of style image paths
        output_base_dir: Base output directory
        batch_name: Batch subfolder name
        steps: Maximum optimization steps per image
        checkpoint_interval: Save checkpoint every N steps
    """

    print("\n" + "="*60)
    print(f"BATCH STYLE TRANSFER")
    print(f"Content images: {len(content_images)}")
    print(f"Style images: {len(style_images)}")
    print(f"Output: {output_base_dir}/{batch_name}/")
    print("="*60)

    for i, content_path in enumerate(content_images, 1):
        # Randomly select a style for this content image
        style_path = random.choice(style_images)

        print(f"\n[{i}/{len(content_images)}] Content: {os.path.basename(content_path)}")
        print(f"           Style: {os.path.basename(style_path)}")

        run_style_transfer(
            content_path=content_path,
            style_path=style_path,
            output_base_dir=output_base_dir,
            batch_name=batch_name,
            steps=steps,
            checkpoint_interval=checkpoint_interval
        )

    print("\n" + "="*60)
    print("✓ ALL TRANSFERS COMPLETE!")
    print("="*60)


# ---------------- DEFINE YOUR IMAGES ----------------

content_images_generated = [
    # DCGAN faces
    "Dataset/generated/dcgan_faces/dcgan_face_1.png",
    "Dataset/generated/dcgan_faces/dcgan_face_2.png",
    "Dataset/generated/dcgan_faces/dcgan_face_3.png",
    # CycleGAN Apple2Orange transformations
    "Dataset/generated/cyclegan_apple2orange/apple2orange_a2b.png",
    "Dataset/generated/cyclegan_apple2orange/apple2orange_b2a.png",
    # CycleGAN Summer2Winter transformations
    "Dataset/generated/cyclegan_summer2winter/summer2winter_a2b.png",
    "Dataset/generated/cyclegan_summer2winter/summer2winter_b2a.png",
]

style_images = [
    "Dataset/NST/nst-picasso.jpg",
    "Dataset/NST/nst-kadinsky.jpg",
]

# Choose which content images to use
content_images = content_images_generated

# Filter out any files that don't exist
content_images = [path for path in content_images if os.path.exists(path)]

print(f"\n📊 Content images to process: {len(content_images)}")
print(f"🎨 Style images available: {len(style_images)}")

# Run batch processing with random style selection
run_batch_style_transfer(
    content_images=content_images,
    style_images=style_images,
    output_base_dir="output_nst",
    batch_name="batch",
    steps=500,
    checkpoint_interval=50
)


📊 Content images to process: 7
🎨 Style images available: 2

BATCH STYLE TRANSFER
Content images: 7
Style images: 2
Output: output_nst/batch/

[1/7] Content: dcgan_face_1.png
           Style: nst-picasso.jpg

Processing: dcgan_face_1 × nst-picasso
Output directory: output_nst\batch\dcgan_face_1__nst-picasso
[  50] Content=4.508502, Style=0.000311
[ 100] Content=3.822603, Style=0.000254
[ 150] Content=3.566385, Style=0.000232
[ 200] Content=3.431097, Style=0.000220
[ 250] Content=3.339631, Style=0.000213
[ 300] Content=3.274787, Style=0.000207
[ 350] Content=3.225577, Style=0.000202
[ 400] Content=3.183739, Style=0.000199
[ 450] Content=3.147371, Style=0.000196
[ 500] Content=3.115758, Style=0.000193
✓ Saved final image: output_nst\batch\dcgan_face_1__nst-picasso/final.png

[2/7] Content: dcgan_face_2.png
           Style: nst-kadinsky.jpg

Processing: dcgan_face_2 × nst-kadinsky
Output directory: output_nst\batch\dcgan_face_2__nst-kadinsky
[  50] Content=26.995358, Style=0.002137
[ 10

In [13]:
# ============================================================================
# APPLY STYLE TRANSFER TO CYCLEGAN PAIRS AND CREATE COMPARISONS
# ============================================================================

def stylize_and_compare_cyclegan_pairs(style_path, output_base_dir="output_nst",
                                       batch_name="cyclegan_comparisons",
                                       steps=500, checkpoint_interval=50):
    """
    Apply style transfer to CycleGAN pair images and create side-by-side comparisons.

    For each model (apple2orange, summer2winter):
    1. Apply style to original image (pair_a)
    2. Apply style to generated image (pair_b)
    3. Concatenate them horizontally for comparison
    """
    import torch

    pairs = [
        {
            "name": "apple2orange",
            "pair_a": "Dataset/generated/cyclegan_apple2orange/apple2orange_pair_a.png",
            "pair_b": "Dataset/generated/cyclegan_apple2orange/apple2orange_pair_b.png",
        },
        {
            "name": "summer2winter",
            "pair_a": "Dataset/generated/cyclegan_summer2winter/summer2winter_pair_a.png",
            "pair_b": "Dataset/generated/cyclegan_summer2winter/summer2winter_pair_b.png",
        }
    ]

    print("\n" + "="*60)
    print("APPLYING STYLE TRANSFER TO CYCLEGAN PAIRS")
    print("="*60)

    for pair_info in pairs:
        name = pair_info["name"]
        pair_a_path = pair_info["pair_a"]
        pair_b_path = pair_info["pair_b"]

        if not os.path.exists(pair_a_path) or not os.path.exists(pair_b_path):
            print(f"\n⚠️  Skipping {name} - files not found")
            continue

        print(f"\n{'='*60}")
        print(f"Processing pair: {name}")
        print(f"{'='*60}")

        # Apply style to original (A)
        print(f"\n[1/2] Stylizing original image...")
        styled_a_tensor = run_style_transfer(
            content_path=pair_a_path,
            style_path=style_path,
            output_base_dir=output_base_dir,
            batch_name=f"{batch_name}/{name}",
            steps=steps,
            checkpoint_interval=checkpoint_interval
        )

        # Apply style to generated (B)
        print(f"\n[2/2] Stylizing generated image...")
        styled_b_tensor = run_style_transfer(
            content_path=pair_b_path,
            style_path=style_path,
            output_base_dir=output_base_dir,
            batch_name=f"{batch_name}/{name}",
            steps=steps,
            checkpoint_interval=checkpoint_interval
        )

        # Load the saved styled images
        styled_a_dir = os.path.join(output_base_dir, f"{batch_name}/{name}",
                                     f"{os.path.splitext(os.path.basename(pair_a_path))[0]}__{os.path.splitext(os.path.basename(style_path))[0]}")
        styled_b_dir = os.path.join(output_base_dir, f"{batch_name}/{name}",
                                     f"{os.path.splitext(os.path.basename(pair_b_path))[0]}__{os.path.splitext(os.path.basename(style_path))[0]}")

        styled_a_img_path = os.path.join(styled_a_dir, "final.png")
        styled_b_img_path = os.path.join(styled_b_dir, "final.png")

        # Load and concatenate
        if os.path.exists(styled_a_img_path) and os.path.exists(styled_b_img_path):
            from torchvision import io
            styled_a_img = io.read_image(styled_a_img_path).float() / 255.0
            styled_b_img = io.read_image(styled_b_img_path).float() / 255.0

            # Concatenate horizontally (side-by-side)
            comparison = torch.cat([styled_a_img, styled_b_img], dim=2)  # dim=2 is width

            # Save comparison
            comparison_dir = os.path.join(output_base_dir, batch_name, name)
            os.makedirs(comparison_dir, exist_ok=True)
            comparison_path = os.path.join(comparison_dir, f"{name}_styled_comparison.png")

            save_image(comparison, comparison_path)
            print(f"\n✅ Saved comparison: {comparison_path}")
            print(f"   (Original styled | Generated styled) - {comparison.shape[2]}x{comparison.shape[1]} px")
        else:
            print(f"\n⚠️  Could not find styled images for comparison")

    print("\n" + "="*60)
    print("✅ ALL COMPARISONS COMPLETE!")
    print("="*60)


# Example usage - uncomment and run when ready:
stylize_and_compare_cyclegan_pairs(
    style_path="Dataset/NST/nst-kadinsky.jpg",
    output_base_dir="output_nst",
    batch_name="cyclegan_comparisons",
    steps=500,
    checkpoint_interval=50
)



APPLYING STYLE TRANSFER TO CYCLEGAN PAIRS

Processing pair: apple2orange

[1/2] Stylizing original image...

Processing: apple2orange_pair_a × nst-kadinsky
Output directory: output_nst\cyclegan_comparisons/apple2orange\apple2orange_pair_a__nst-kadinsky
[  50] Content=25.404797, Style=0.002440
[ 100] Content=18.381714, Style=0.001537
[ 150] Content=15.861139, Style=0.001259
[ 200] Content=14.541812, Style=0.001135
[ 250] Content=13.725191, Style=0.001065
[ 300] Content=13.182171, Style=0.001020
[ 350] Content=12.791217, Style=0.000988
[ 400] Content=12.479332, Style=0.000966
[ 450] Content=12.229878, Style=0.000948
[ 500] Content=12.028555, Style=0.000934
✓ Saved final image: output_nst\cyclegan_comparisons/apple2orange\apple2orange_pair_a__nst-kadinsky/final.png

[2/2] Stylizing generated image...

Processing: apple2orange_pair_b × nst-kadinsky
Output directory: output_nst\cyclegan_comparisons/apple2orange\apple2orange_pair_b__nst-kadinsky
[  50] Content=27.426661, Style=0.003057
[ 10

In [11]:
# ============================================================================
# GENERATE IMAGES FROM TRAINED MODELS FOR NST CONTENT INPUT
# ============================================================================
import torch
import os
from torchvision.utils import save_image
from torchvision import transforms
from PIL import Image

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ---------------- DCGAN IMAGE GENERATION ----------------
def generate_dcgan_images(checkpoint_dir, output_dir, num_samples=3, noise_size=100):
    """
    Generate 3 random face images from trained DCGAN model.

    Outputs:
        - dcgan_face_1.png
        - dcgan_face_2.png
        - dcgan_face_3.png
    """
    from Model import DCGAN_Generator, load_checkpoint_generic

    print(f"\n{'='*60}")
    print(f"GENERATING DCGAN IMAGES")
    print(f"{'='*60}")

    os.makedirs(output_dir, exist_ok=True)

    # Load checkpoint
    checkpoint = load_checkpoint_generic(checkpoint_dir, device)
    if not checkpoint:
        print(f"❌ No checkpoint found in {checkpoint_dir}")
        return []

    # Initialize generator
    generator = DCGAN_Generator(z_dim=noise_size, img_channels=3).to(device)
    generator.load_state_dict(checkpoint['generator'])
    generator.eval()

    print(f"✅ Loaded generator from epoch {checkpoint.get('epoch', '?')}")

    generated_files = []

    # Generate 3 random face images
    with torch.no_grad():
        for i in range(num_samples):
            noise = torch.randn(1, noise_size, 1, 1, device=device)
            fake_img = generator(noise)

            # Denormalize from [-1, 1] to [0, 1]
            fake_img = (fake_img + 1) / 2.0

            # Upscale to 1024x1024 for NST
            upscale = transforms.Resize((1024, 1024), interpolation=transforms.InterpolationMode.BICUBIC)
            fake_img_large = upscale(fake_img)

            output_path = os.path.join(output_dir, f"dcgan_face_{i+1}.png")
            save_image(fake_img_large, output_path)
            generated_files.append(output_path)
            print(f"   [{i+1}/3] Saved: {output_path}")

    print(f"✅ Generated {num_samples} DCGAN face images")
    del generator
    torch.cuda.empty_cache()

    return generated_files


# ---------------- CycleGAN IMAGE GENERATION ----------------
def generate_cyclegan_images(checkpoint_dir, dataset_root, output_dir, model_name="cyclegan"):
    """
    Generate 3 content images from trained CycleGAN model:
    1. A→B transformation (single output)
    2. B→A transformation (single output)
    3. A→B pair (original A + generated B side-by-side for comparison)

    Outputs:
        - cyclegan_a2b.png       (transformed A→B)
        - cyclegan_b2a.png       (transformed B→A)
        - cyclegan_pair_a.png    (original A from testA)
        - cyclegan_pair_b.png    (transformed A→B for comparison)
    """
    from Model import CycleGAN_Generator, load_checkpoint_generic

    print(f"\n{'='*60}")
    print(f"GENERATING CycleGAN IMAGES - {model_name}")
    print(f"{'='*60}")

    os.makedirs(output_dir, exist_ok=True)

    # Load checkpoint
    checkpoint = load_checkpoint_generic(checkpoint_dir, device)
    if not checkpoint:
        print(f"❌ No checkpoint found in {checkpoint_dir}")
        return []

    # Initialize both generators
    G_A2B = CycleGAN_Generator().to(device)
    G_B2A = CycleGAN_Generator().to(device)

    G_A2B.load_state_dict(checkpoint['G_A2B'])
    G_B2A.load_state_dict(checkpoint['G_B2A'])

    G_A2B.eval()
    G_B2A.eval()

    print(f"✅ Loaded generators from epoch {checkpoint.get('epoch', '?')}")

    # Load test images
    test_dir_A = os.path.join(dataset_root, 'testA')
    test_dir_B = os.path.join(dataset_root, 'testB')

    test_images_A = sorted([f for f in os.listdir(test_dir_A) if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
    test_images_B = sorted([f for f in os.listdir(test_dir_B) if f.lower().endswith(('.jpg', '.jpeg', '.png'))])

    if len(test_images_A) == 0 or len(test_images_B) == 0:
        print(f"❌ No test images found in {test_dir_A} or {test_dir_B}")
        return []

    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]),
    ])

    upscale = transforms.Resize((1024, 1024), interpolation=transforms.InterpolationMode.BICUBIC)

    generated_files = []

    with torch.no_grad():
        # 1. Generate A→B (single output)
        print(f"\n[1/3] Generating A→B transformation...")
        img_path_A = os.path.join(test_dir_A, test_images_A[0])
        img_A = Image.open(img_path_A).convert('RGB')
        img_A_tensor = transform(img_A).unsqueeze(0).to(device)

        fake_B = G_A2B(img_A_tensor)
        fake_B = (fake_B + 1) / 2.0  # Denormalize
        fake_B_large = upscale(fake_B)

        output_a2b = os.path.join(output_dir, f"{model_name}_a2b.png")
        save_image(fake_B_large, output_a2b)
        generated_files.append(output_a2b)
        print(f"   Saved: {output_a2b}")

        # 2. Generate B→A (single output)
        print(f"\n[2/3] Generating B→A transformation...")
        img_path_B = os.path.join(test_dir_B, test_images_B[0])
        img_B = Image.open(img_path_B).convert('RGB')
        img_B_tensor = transform(img_B).unsqueeze(0).to(device)

        fake_A = G_B2A(img_B_tensor)
        fake_A = (fake_A + 1) / 2.0  # Denormalize
        fake_A_large = upscale(fake_A)

        output_b2a = os.path.join(output_dir, f"{model_name}_b2a.png")
        save_image(fake_A_large, output_b2a)
        generated_files.append(output_b2a)
        print(f"   Saved: {output_b2a}")

        # 3. Generate A→B pair (original A + generated B for comparison)
        print(f"\n[3/3] Generating A→B comparison pair...")
        # Use a different image from testA for variety
        img_path_A_pair = os.path.join(test_dir_A, test_images_A[1 % len(test_images_A)])
        img_A_pair = Image.open(img_path_A_pair).convert('RGB')
        img_A_pair_tensor = transform(img_A_pair).unsqueeze(0).to(device)

        # Original A
        img_A_pair_norm = (img_A_pair_tensor + 1) / 2.0  # Denormalize
        img_A_pair_large = upscale(img_A_pair_norm)

        # Generated B from A
        fake_B_pair = G_A2B(img_A_pair_tensor)
        fake_B_pair = (fake_B_pair + 1) / 2.0  # Denormalize
        fake_B_pair_large = upscale(fake_B_pair)

        # Save both separately for individual style transfer
        output_pair_a = os.path.join(output_dir, f"{model_name}_pair_a.png")
        output_pair_b = os.path.join(output_dir, f"{model_name}_pair_b.png")

        save_image(img_A_pair_large, output_pair_a)
        save_image(fake_B_pair_large, output_pair_b)

        generated_files.append(output_pair_a)
        generated_files.append(output_pair_b)

        print(f"   Saved original A: {output_pair_a}")
        print(f"   Saved generated B: {output_pair_b}")
        print(f"   (These will be styled separately then concatenated for comparison)")

    print(f"\n✅ Generated 4 CycleGAN images (2 singles + 1 pair)")
    del G_A2B, G_B2A
    torch.cuda.empty_cache()

    return generated_files


# ============================================================================
# MAIN EXECUTION - Generate all images
# ============================================================================

all_generated_files = []

# 1. DCGAN - Generate 3 random faces
print("\n" + "🎨 "*30)
dcgan_files = generate_dcgan_images(
    checkpoint_dir="output_dcgan",
    output_dir="Dataset/generated/dcgan_faces",
    num_samples=3,
    noise_size=100
)
all_generated_files.extend(dcgan_files)

# 2. CycleGAN - Apple to Orange (1 A→B, 1 B→A, 1 pair)
print("\n" + "🍎 "*30)
a2o_files = generate_cyclegan_images(
    checkpoint_dir="output_cyclegan_a2o",
    dataset_root="Dataset/Apple2orange",
    output_dir="Dataset/generated/cyclegan_apple2orange",
    model_name="apple2orange"
)
all_generated_files.extend(a2o_files)

# 3. CycleGAN - Summer to Winter (1 A→B, 1 B→A, 1 pair)
print("\n" + "❄️ "*30)
s2w_files = generate_cyclegan_images(
    checkpoint_dir="output_cyclegan_s2w",
    dataset_root="Dataset/Summer2Winter_Yosemite",
    output_dir="Dataset/generated/cyclegan_summer2winter",
    model_name="summer2winter"
)
all_generated_files.extend(s2w_files)
#
# # ============================================================================
# # SUMMARY
# # ============================================================================
print("\n" + "="*60)
print("✅ ALL IMAGE GENERATION COMPLETE!")
print("="*60)
print(f"\nTotal files generated: {len(all_generated_files)}")



🎨 🎨 🎨 🎨 🎨 🎨 🎨 🎨 🎨 🎨 🎨 🎨 🎨 🎨 🎨 🎨 🎨 🎨 🎨 🎨 🎨 🎨 🎨 🎨 🎨 🎨 🎨 🎨 🎨 🎨 

GENERATING DCGAN IMAGES
✅ Loaded checkpoint: output_dcgan\checkpoint_epoch_500.pth (epoch 500)
✅ Loaded generator from epoch 500
   [1/3] Saved: Dataset/generated/dcgan_faces\dcgan_face_1.png
   [2/3] Saved: Dataset/generated/dcgan_faces\dcgan_face_2.png
   [3/3] Saved: Dataset/generated/dcgan_faces\dcgan_face_3.png
✅ Generated 3 DCGAN face images

🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 🍎 

GENERATING CycleGAN IMAGES - apple2orange
✅ Loaded checkpoint: output_cyclegan_a2o\checkpoint_epoch_50.pth (epoch 50)
✅ Loaded generators from epoch 50

[1/3] Generating A→B transformation...
   Saved: Dataset/generated/cyclegan_apple2orange\apple2orange_a2b.png

[2/3] Generating B→A transformation...
   Saved: Dataset/generated/cyclegan_apple2orange\apple2orange_b2a.png

[3/3] Generating A→B comparison pair...
   Saved original A: Dataset/generated/cyclegan_apple2orange\apple2orange_pair_a.png
   Saved generated B: Da

# V3.1 DCGAN output

<img src="output_dcgan/loss_curves.png"/>

<!-- DCGAN Training Progress - Selected Epochs -->
<div style="display: flex; justify-content: space-around;">
  <div style="text-align: center;">
    <img src="output_dcgan/epoch_0100.png" width="180"/><br>
    <small>Epoch 100</small>
  </div>
  <div style="text-align: center;">
    <img src="output_dcgan/epoch_0200.png" width="180"/><br>
    <small>Epoch 200</small>
  </div>
  <div style="text-align: center;">
    <img src="output_dcgan/epoch_0300.png" width="180"/><br>
    <small>Epoch 300</small>
  </div>
  <div style="text-align: center;">
    <img src="output_dcgan/epoch_0400.png" width="180"/><br>
    <small>Epoch 400</small>
  </div>
  <div style="text-align: center;">
    <img src="output_dcgan/epoch_0500.png" width="180"/><br>
    <small>Epoch 500</small>
  </div>
</div>

<img src="output_dcgan/training_progress.gif" width="1000"/>


# V3.2 CycleGAN - Apple2Orange

## Loss Curves
<img src="output_cyclegan_a2o/loss_curves.png"/>

## Training Progress - Selected Epochs
<!-- Apple2Orange CycleGAN Training Progress -->
<div style="display: flex; justify-content: space-around;">
  <div style="text-align: center;">
    <img src="output_cyclegan_a2o/epoch_0010.png" width="180"/><br>
    <small>Epoch 10</small>
  </div>
  <div style="text-align: center;">
    <img src="output_cyclegan_a2o/epoch_0020.png" width="180"/><br>
    <small>Epoch 20</small>
  </div>
  <div style="text-align: center;">
    <img src="output_cyclegan_a2o/epoch_0030.png" width="180"/><br>
    <small>Epoch 30</small>
  </div>
  <div style="text-align: center;">
    <img src="output_cyclegan_a2o/epoch_0040.png" width="180"/><br>
    <small>Epoch 40</small>
  </div>
  <div style="text-align: center;">
    <img src="output_cyclegan_a2o/epoch_0050.png" width="180"/><br>
    <small>Epoch 50</small>
  </div>
</div>

## Training Animation
<img src="output_cyclegan_a2o/training_progress_50.gif" width="800"/>

## Generated Samples

### Apple → Orange Transformation
<div style="display: flex; justify-content: space-around; align-items: center;">
  <div style="text-align: center;">
    <img src="Dataset/generated/cyclegan_apple2orange/apple2orange_pair_a.png" width="300"/><br>
    <small>Original Apple</small>
  </div>
  <div style="text-align: center; font-size: 48px;">→</div>
  <div style="text-align: center;">
    <img src="Dataset/generated/cyclegan_apple2orange/apple2orange_pair_b.png" width="300"/><br>
    <small>Generated Orange</small>
  </div>
</div>

### Orange → Apple Transformation
<div style="text-align: center;">
  <img src="Dataset/generated/cyclegan_apple2orange/apple2orange_b2a.png" width="300"/><br>
  <small>Orange → Apple</small>
</div>

### Single Transformation Output
<div style="text-align: center;">
  <img src="Dataset/generated/cyclegan_apple2orange/apple2orange_a2b.png" width="300"/><br>
  <small>Apple → Orange (Single Output)</small>
</div>